In [ ]:
import pandas as pd
import re

In [6]:
#
# todo 걍 토픽분석이나 하셈 ㅠ
# 아니 근데 토픽분석의 의미가 있나? 무슨 뉴스 키워드 검색도 아니고,
# 각 영화의 해.설.인데.. 각 영화 주제도 다양할 테고,
# 특정 주제를 잡고 한 것도 아니라서 먼가 그렇긴 한데.. 일단 연습삼아 해보자
    # 그래서 일단 type을 나누려고 하긴 했는데, type도 movie & tv_show 2개밖에 없음
    # ^_^.. 처음부터 주제를 제대로 잡고 했었어야 했음

In [2]:
netflix_df = pd.read_csv('4.4ref/netflix_titles.csv', encoding ='utf8', sep = ',', engine = 'python')

# len(netflix_df) # 7787
netflix_df2 = netflix_df[['type', 'description']]

wrong_way = netflix_df2.groupby('type') # 이렇게 하면 안됨. 뒤에 count()붙이거나 하면 되긴 하는데 모든 데이터 그룹화는 x
wrong_way # <pandas.core.groupby.generic.DataFrameGroupBy object at 0x0000015352539010>

netflix_typed = netflix_df2.groupby('type').apply(lambda x: x)
netflix_typed

C:\Users\A\AppData\Local\Temp\ipykernel_9212\4264831729.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  netflix_typed = netflix_df2.groupby('type').apply(lambda x: x)


type                                        description
type                                                                    
Movie   1       Movie  After a devastating earthquake hits Mexico Cit...
        2       Movie  When an army recruit is found dead, his fellow...
        3       Movie  In a postapocalyptic world, rag-doll robots hi...
        4       Movie  A brilliant group of students become card-coun...
        6       Movie  After an awful accident, a couple admitted to ...
...               ...                                                ...
TV Show 7767  TV Show  Strong-willed, middle-class Kashaf and carefre...
        7775  TV Show  A quest for freedom and legendary treasure beg...
        7777  TV Show  While living alone in a spooky town, a young g...
        7779  TV Show  An assortment of talent takes the stage for a ...
        7785  TV Show  Dessert wizard Adriano Zumbo looks for the nex...

[7787 rows x 2 columns]

In [4]:
# 결측치 제거
netflix_typed.info() # 정보확인
netflix_typed = netflix_typed[netflix_typed['description'].notnull()] # 결측치 제거 - 인데, 원래 없음

netflix_typed['type'].value_counts()
# Movie: 5377 / TV Show: 2410

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7787 entries, ('Movie', 1) to ('TV Show', 7785)
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   type         7787 non-null   object
 1   description  7787 non-null   object
dtypes: object(2)
memory usage: 463.5+ KB


type
Movie      5377
TV Show    2410
Name: count, dtype: int64

In [5]:
# 쓸데 없는 문자 제거 후에, 타입별로 나누기
netflix_typed['description'] = netflix_typed['description'].apply(lambda x : re.sub(r"[^a-zA-Z]", " ", x))
netflix_typed

# movie / tv_show
netflix_movie = netflix_typed[netflix_typed['type'] == 'Movie'][['description']]
netflix_tvshow = netflix_typed[netflix_typed['type'] == 'TV Show'][['description']]
netflix_movie # 5377
netflix_tvshow # 2410

description
type                                                           
TV Show 0     In a future where the elite inhabit an island ...
        5     A genetics professor experiments with a treatm...
        11    In this dark alt history thriller  a na ve law...
        12    Archival video and new interviews examine Mexi...
        16    As a psychology professor faces Alzheimer s  h...
...                                                         ...
        7767  Strong willed  middle class Kashaf and carefre...
        7775  A quest for freedom and legendary treasure beg...
        7777  While living alone in a spooky town  a young g...
        7779  An assortment of talent takes the stage for a ...
        7785  Dessert wizard Adriano Zumbo looks for the nex...

[2410 rows x 1 columns]

In [6]:
import nltk
from nltk.tokenize import word_tokenize
# nltk.download('punkt') # 최초 1번만 다운

# 영어 문장을 토큰화하기 위한 함수를 정의
def nltk_tokenizer(text):
    tokens = word_tokenize(text)
    return tokens

In [8]:
movie_description_noun_tk = []

for d in netflix_movie['description']:
    noun_tokens = nltk_tokenizer(d)
    noun_tokens2 = [word for word in noun_tokens if len(word)>1]
    movie_description_noun_tk.append(noun_tokens2)

movie_description_noun_tk

[['After',
  'devastating',
  'earthquake',
  'hits',
  'Mexico',
  'City',
  'trapped',
  'survivors',
  'from',
  'all',
  'walks',
  'of',
  'life',
  'wait',
  'to',
  'be',
  'rescued',
  'while',
  'trying',
  'desperately',
  'to',
  'stay',
  'alive'],
 ['When',
  'an',
  'army',
  'recruit',
  'is',
  'found',
  'dead',
  'his',
  'fellow',
  'soldiers',
  'are',
  'forced',
  'to',
  'confront',
  'terrifying',
  'secret',
  'that',
  'haunting',
  'their',
  'jungle',
  'island',
  'training',
  'camp'],
 ['In',
  'postapocalyptic',
  'world',
  'rag',
  'doll',
  'robots',
  'hide',
  'in',
  'fear',
  'from',
  'dangerous',
  'machines',
  'out',
  'to',
  'exterminate',
  'them',
  'until',
  'brave',
  'newcomer',
  'joins',
  'the',
  'group'],
 ['brilliant',
  'group',
  'of',
  'students',
  'become',
  'card',
  'counting',
  'experts',
  'with',
  'the',
  'intent',
  'of',
  'swindling',
  'millions',
  'out',
  'of',
  'Las',
  'Vegas',
  'casinos',
  'by',
  'pla

In [11]:
import gensim
import gensim.corpora as corpora


# 1. description_noun_tk2에 포함된 단어에 대해 사전을 구성corpora.Dictionary( )
dictionary = corpora.Dictionary(movie_description_noun_tk)
dictionary # print해야 값 나옴
print(dictionary) # Dictionary<16055 unique tokens: ['After', 'City', 'Mexico', 'alive', 'all']...>

# 2. 단어 사전dictionary의 단어에 대해 BoW를 구하여doc2bow( ), 단어 뭉치corpus를 구성
    # (word_id, work_count)의 BoW 구성
corpus = [dictionary.doc2bow(word) for word in movie_description_noun_tk]
print(corpus)

Dictionary<16055 unique tokens: ['After', 'City', 'Mexico', 'alive', 'all']...>
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1)], [(16, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1)], [(9, 1), (16, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1)], [(12, 3), (50, 1), (56, 1), (60, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1)], [(0, 1), (16, 2), (23, 1), (24, 1), (61, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90,

In [12]:
# 3. LDA 토픽 모델의 생성 및 훈련하기 - 토픽개수 4개
    # gensim 패키지의 LDA 모듈을 이용하여 토픽 모델 객체인 lda_model을 생성

k = 4 # 토픽의 개수 설정
lda_model = gensim.models.ldamulticore.LdaMulticore(
    corpus, iterations=12, num_topics = k, id2word = dictionary, passes = 1, workers = 10)

In [13]:
print(lda_model.print_topics(num_topics = k, num_words = 15))

[(0, '0.029*"the" + 0.028*"to" + 0.026*"and" + 0.020*"his" + 0.018*"of" + 0.016*"in" + 0.015*"her" + 0.010*"with" + 0.009*"an" + 0.008*"for" + 0.008*"on" + 0.007*"he" + 0.005*"is" + 0.005*"their" + 0.005*"from"'), (1, '0.033*"to" + 0.032*"the" + 0.026*"and" + 0.020*"of" + 0.019*"his" + 0.014*"in" + 0.010*"with" + 0.008*"an" + 0.008*"her" + 0.008*"for" + 0.007*"their" + 0.007*"from" + 0.006*"on" + 0.006*"he" + 0.005*"who"'), (2, '0.034*"the" + 0.029*"of" + 0.026*"to" + 0.022*"and" + 0.020*"in" + 0.010*"her" + 0.010*"his" + 0.009*"with" + 0.008*"their" + 0.007*"for" + 0.006*"an" + 0.006*"on" + 0.006*"is" + 0.005*"from" + 0.005*"by"'), (3, '0.036*"the" + 0.031*"to" + 0.027*"and" + 0.025*"of" + 0.020*"his" + 0.017*"in" + 0.010*"for" + 0.010*"on" + 0.009*"with" + 0.008*"an" + 0.008*"their" + 0.007*"her" + 0.006*"is" + 0.005*"who" + 0.005*"that"')]


In [14]:
import os
import pyLDAvis.gensim

# 한글 UnicodeEncodeError 방지를 위해 기본 인코딩을 "utf-8"로 설정
os.environ["PYTHONIOENCODING"] = "utf-8"

# LDA 토픽 분석의 결과를 시각화하기 위해 pyLDAvis 패키지의 pyLDAvis.gensim.prepare() 함수를 사용
lda_vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)

pyLDAvis.display(lda_vis)

In [ ]:
# 어 근데 뽑고나니까, 상위30개 단위중에 "명사"는 5개에 불과하였음
# todo -> 관사, 인칭대명사같은거 빼고 다시 데이터 추출해야 함 ㅠ
# -> 전처리 과정중에 아래 방법 중에 하나를 선택해서 추가하면 된다

In [ ]:
# ! pip install spacy
# python -m spacy download en # + 이것도 bash에서 설치해야 함

In [35]:
# Way1: 영어에서 명사 추출
import spacy

nlp = spacy.load("en_core_web_sm") # en_core_web_sm는 영어에 대한 기본 모델

# 주어진 텍스트에서 명사만 추출하여 반환합니다 (spaCy 사용)
def extract_nouns_spacy(text):
    doc = nlp(text)
    nouns = [token.text for token in doc if token.pos_ == "NOUN"]
    return nouns

ex = 'noun the a they it' # ['a']
ex2 = 'i want to buy ticket' # ['ticket']
ex3 = 'word noun' # ['word']
extract_nouns_spacy(ex)

# * -> 어이구;; way2가 훨씬 낫네. 너무 이상하게 걸러준다 spacy는

['a']

In [41]:
# Way2: 영어에서 명사만 추출하는 방법
    # * 그런데, 명사만 추출해야 하는가? 'I', 'you' 이런것도 중요한 키워드가 아닐까?
    # "키워드"를 어떻게 잡는가? -> 이것도 나중에 고려해봐야 할듯

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
# nltk.download('punkt') # 토큰화를 위한 리소스, 최초 실행 시 필요
# nltk.download('averaged_perceptron_tagger') # 품사 태깅을 위한 리소스, 최초 실행 시 필요

# 주어진 텍스트에서 명사만 추출하여 반환
def extract_nouns(text):
    
    tokens = word_tokenize(text) # 텍스트를 토큰화
    tagged_tokens = pos_tag(tokens) # 토큰에 품사 태깅을 수행
    # 명사 품사 태그를 가진 토큰만 추출
    nouns = [word for word, tag in tagged_tokens if tag in ("NN", "NNS", "NNP", "NNPS")]
    
    return nouns

ex = 'noun the a they it' # []
ex2 = 'i want to buy ticket' # ['i', 'ticket']
ex3 = 'word noun' # ['word', 'noun']
ex4 = 'his and love the worlds'
extract_nouns(ex4)

['worlds']

In [38]:
movie_description_noun_tk_re = []

for d in netflix_movie['description']:
    # noun_tokens = nltk_tokenizer(d) # 필요없음
    noun_tokens = extract_nouns(d) # 명사를 걸러줌"
    noun_tokens2 = [word for word in noun_tokens if len(word)>1]
    movie_description_noun_tk_re.append(noun_tokens2)

movie_description_noun_tk_re

[['earthquake',
  'hits',
  'Mexico',
  'City',
  'survivors',
  'walks',
  'life',
  'wait'],
 ['army',
  'recruit',
  'soldiers',
  'terrifying',
  'secret',
  'jungle',
  'island',
  'training',
  'camp'],
 ['world', 'rag', 'doll', 'robots', 'fear', 'machines', 'newcomer', 'group'],
 ['group',
  'students',
  'counting',
  'experts',
  'intent',
  'millions',
  'Las',
  'Vegas',
  'casinos',
  'blackjack'],
 ['accident', 'couple', 'hospital', 'death'],
 ['school',
  'students',
  'teacher',
  'Trevor',
  'Garfield',
  'weary',
  'gang',
  'warfare',
  'New',
  'York',
  'City',
  'school',
  'system',
  'moves',
  'California',
  'environment'],
 ['doctor', 'psychiatrist', 'wife', 'condition', 'patient'],
 ['architect', 'wife', 'move', 'castle', 'hotel', 'something', 'renovation'],
 ['farmer',
  'confession',
  'wife',
  'murder',
  'death',
  'beginning',
  'tale',
  'Stephen',
  'King',
  'novella'],
 ['documentary',
  'rock',
  'star',
  'Artiwara',
  'Kongmalai',
  'kilometer',


In [42]:
import gensim
import gensim.corpora as corpora


# 1. description_noun_tk2에 포함된 단어에 대해 사전을 구성corpora.Dictionary( )
dictionary = corpora.Dictionary(movie_description_noun_tk_re)
dictionary # print해야 값 나옴
print(dictionary) # Dictionary<16055 unique tokens: ['After', 'City', 'Mexico', 'alive', 'all']...>

# 2. 단어 사전dictionary의 단어에 대해 BoW를 구하여doc2bow( ), 단어 뭉치corpus를 구성
    # (word_id, work_count)의 BoW 구성
corpus = [dictionary.doc2bow(word) for word in movie_description_noun_tk_re]
print(corpus)

# 3. LDA 토픽 모델의 생성 및 훈련하기 - 토픽개수 4개
    # gensim 패키지의 LDA 모듈을 이용하여 토픽 모델 객체인 lda_model을 생성

k = 4 # 토픽의 개수 설정
lda_model = gensim.models.ldamulticore.LdaMulticore(
    corpus, iterations=12, num_topics = k, id2word = dictionary, passes = 1, workers = 10)

print(lda_model.print_topics(num_topics = k, num_words = 15))

Dictionary<10996 unique tokens: ['City', 'Mexico', 'earthquake', 'hits', 'life']...>
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1)], [(17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1)], [(19, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1)], [(34, 1), (35, 1), (36, 1), (37, 1)], [(0, 1), (33, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 2), (47, 1), (48, 1), (49, 1), (50, 1)], [(51, 1), (52, 1), (53, 1), (54, 1), (55, 1)], [(55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1)], [(36, 1), (55, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1)], [(70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1)], [(78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1)], [(84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1)], [(92, 1), (93, 1), 

In [43]:
import os
import pyLDAvis.gensim

# 한글 UnicodeEncodeError 방지를 위해 기본 인코딩을 "utf-8"로 설정
os.environ["PYTHONIOENCODING"] = "utf-8"

# LDA 토픽 분석의 결과를 시각화하기 위해 pyLDAvis 패키지의 pyLDAvis.gensim.prepare() 함수를 사용
lda_vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)

pyLDAvis.display(lda_vis) # 어찌되었든 유의미한 결과가 나왔다고 생각!